# Market Basket

In [1]:
## Import das bibliotecas
import pandas as pd
import pandasql as ps
import sqlalchemy as sq
import urllib
import findspark
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import collect_set, col, count
from pyspark.sql import SparkSession

## Pandas Configuration
pd.options.display.max_rows    = 999
pd.options.display.max_columns = 999

In [2]:
spark

In [3]:
path = r'C:\Github\MarketBasketAnalysis\Dataset'
dataset = pd.read_csv(path+r'\dataset.csv',sep=';')
clientes = pd.read_csv(path+r'\clientes.csv',sep=';')
grupo_cliente = pd.read_csv(path+r'\grupo_cliente.csv',sep=';')
itens = pd.read_csv(path+r'\itens.csv',sep=';')

In [4]:
## Query das tabelas
query_tabelas = """
SELECT
    ds.id_compra,
    ds.cd_compra,
    it.cd_item,
    it.tx_item,
    cl.cd_cliente,
    cl.cliente_nome,
    gc.cd_grupo_cliente,
    gc.cd_pais,
    gc.tx_pais,
    gc.cd_estado,
    ds.dt_compra,
    ds.nm_quantidade,
    ds.nm_vl_item,
    ds.nm_vl_total	
FROM
    dataset ds
LEFT JOIN
    clientes cl
ON
    cl.id_cliente = ds.id_cliente
LEFT JOIN
    grupo_cliente gc
ON
    gc.id_grupo = ds.id_grupo
LEFT JOIN
    itens it
ON
    it.id_item = ds.id_item
WHERE
    gc.cd_grupo_cliente in (0,1,2,3,4,6,7,10,19,22)
"""

df = ps.sqldf(query_tabelas)
df

,id_compra,cd_compra,cd_item,tx_item,cd_cliente,cliente_nome,cd_grupo_cliente,cd_pais,tx_pais,cd_estado,dt_compra,nm_quantidade,nm_vl_item,nm_vl_total
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.95,83.4
1,1,489434,79323P,PINK CHERRY LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.75,81.0
2,2,489434,79323W,WHITE CHERRY LIGHTS,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,12,6.75,81.0
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,48,2.10,100.8
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,13085,James,0,GB,United Kingdom,"London, City of",2009-12-01,24,1.25,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96507,148098,536858,22554,PLASTERS IN TIN WOODLAND ANIMALS,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,36,1.65,59.4
96508,148099,536858,21731,RED TOADSTOOL LED NIGHT LIGHT,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,24,1.65,39.6
96509,148100,536858,20677,PINK SPOTTY BOWL,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,16,1.25,20.0
96510,148101,536858,20750,RED RETROSPOT MINI CASES,13520,Glenda,22,GB,United Kingdom,Wolverhampton,2010-12-03,2,7.95,15.9


In [5]:
## Tratamento dos dados
grupos_query = """
SELECT DISTINCT
    cd_grupo_cliente,
    cd_estado as tx_grupo_cliente
FROM
    df
ORDER BY
    1    
"""
grupos = ps.sqldf(grupos_query)
grupos['tx_grupo_cliente'] = grupos['tx_grupo_cliente'].replace(', City of','',regex=True).replace([" ","-"],"_",regex=True).str.lower()#.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
list_grupos = grupos['cd_grupo_cliente'].values
grupos

,cd_grupo_cliente,tx_grupo_cliente
0,0,london
1,1,birmingham
2,2,manchester
3,3,leeds
4,4,newcastle_upon_tyne
5,6,glasgow_city
6,7,liverpool
7,10,nottingham
8,19,belfast
9,22,wolverhampton


In [6]:
baskets = []
sparkdf = spark.createDataFrame(df)
sparkdf.registerTempTable("spdf")
for i in list_grupos:
    bask = 'basket_' + grupos.loc[grupos['cd_grupo_cliente'] == i,'tx_grupo_cliente'].values[0]
    raw = spark.sql(f"SELECT                       \
                        cd_cliente,                \
                        tx_item                    \
                    FROM                           \
                        spdf                       \
                    WHERE                          \
                        cd_grupo_cliente = {i} AND \
                        nm_quantidade >= 1         \
                    ORDER BY                       \
                        cd_cliente,               \
                        tx_item                    ")
    globals()[bask] = raw.groupBy(['cd_cliente']).agg(collect_set('tx_item').alias('items')).select(['cd_cliente','items']).repartition(1000)
    baskets.append(bask)
print(baskets)

['basket_london', 'basket_birmingham', 'basket_manchester', 'basket_leeds', 'basket_newcastle_upon_tyne', 'basket_glasgow_city', 'basket_liverpool', 'basket_nottingham', 'basket_belfast', 'basket_wolverhampton']


In [ ]:
basket_london.show()

In [ ]:
models = []
fp_growth = FPGrowth(itemsCol='items',minSupport=0.01,minConfidence=0.4,numPartitions=1000)
for i in baskets:
    mdl = 'model_' + i.replace('basket_','')
    models.append(mdl)
    print(f"--------------------------------------------\nModel {mdl} fit - Begin")
    globals()[i].createOrReplaceTempView('baskets')
    bask_tmp = spark.sql("select items from baskets")
    globals()[mdl] = fp_growth.fit(bask_tmp)
    print(f"Model {mdl} fit - End\n--------------------------------------------")
print(models)

--------------------------------------------
Model model_london fit - Begin


In [ ]:
for i in models:
    print('saving ' + i + ' begin')
    globals()[i].write().overwrite().save(r'C:\Github\MarketBasketAnalysis'+f'\\Modelos\\{i}')
    print('saving ' + i + ' end\n')